In [13]:
import json
import os
import pickle
import warnings

import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import pandas as pd

from utils.data_utils import (DynamicBatchSampler, collate_fn, get_dataset,
                              get_grouping, get_static_dataset,
                              get_test_dataset, get_test_synthetic_dataset)
from utils.optuna_utils import load_best_model
from utils.result_utils import (inference, plot_3d_combined_pdfs, plot_pdf,
                                report_results)
from utils.train_utils import ComparisonQuantileLoss, TwoStageQuantileLoss, train
from LinearRegression import LinearQuantileRegression

with open("config.json", "r") as f:
    config = json.load(f)

quantiles = config["general"]["quantiles"]
validation_start_date = config["general"]["dates"]["validation_period"]["start_date"]
validation_end_date = config["general"]["dates"]["validation_period"]["end_date"]
test_start_date = config["general"]["dates"]["test_period"]["start_date"]
test_end_date = config["general"]["dates"]["test_period"]["end_date"]
loss_fn = TwoStageQuantileLoss(quantiles)
test_loss_fn = ComparisonQuantileLoss(quantiles)
results = {}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

warnings.filterwarnings("ignore")

if not os.path.exists("plots"):
    os.makedirs("plots")

In [ ]:
lstm_model, lstm_params = load_best_model('lstm')

lstm_model.to(DEVICE)

lstm_normalization_window = lstm_params['normalazation_window']
lstm_batch_size = lstm_params['batch_size']
l1_reg = lstm_params['l1_reg']
l2_reg = lstm_params['l2_reg']

lstm_optimizer = torch.optim.Adam(lstm_model.parameters(
), lr=lstm_params['learning_rate'], weight_decay=l2_reg)

print(lstm_model)
print(f"Model has {sum(p.numel() for p in lstm_model.parameters() if p.requires_grad)} parameters")

In [ ]:
lstm_params

In [ ]:
if os.path.exists('lstm_model.pth'):
    lstm_model.load_state_dict(torch.load('lstm_model.pth'))
    lstm_model.to(DEVICE)
    print("Model loaded from lstm_model.pth")
else:
    lstm_train_dataset = get_dataset(
        lstm_normalization_window, "1998-01-01", validation_start_date)
    lstm_val_dataset = get_dataset(lstm_normalization_window,
                                   validation_start_date, validation_end_date)
    lstm_train_batch_sampler = DynamicBatchSampler(
        lstm_train_dataset, batch_size=lstm_batch_size)
    lstm_val_batch_sampler = DynamicBatchSampler(lstm_val_dataset, batch_size=lstm_batch_size)

    lstm_train_loader = DataLoader(
        lstm_train_dataset, batch_sampler=lstm_train_batch_sampler, collate_fn=collate_fn)
    lstm_val_loader = DataLoader(
        lstm_val_dataset, batch_sampler=lstm_val_batch_sampler, collate_fn=collate_fn)
    _, lstm_model = train(
        model=lstm_model,
        train_loader=lstm_train_loader,
        val_loader=lstm_val_loader,
        criterion=loss_fn,
        optimizer=lstm_optimizer,
        num_epochs=100,
        patience=10,
        l1_reg=l1_reg,
        lstm=True,
        verbose=True
    )
    torch.save(lstm_model.state_dict(), 'lstm_model.pth')

In [ ]:
dense_model, dense_params = load_best_model('dense')

dense_model.to(DEVICE)

dense_normalization_window = dense_params['normalazation_window']
dense_batch_size = dense_params['batch_size']
l1_reg = dense_params['l1_reg']
l2_reg = dense_params['l2_reg']

dense_optimizer = torch.optim.Adam(dense_model.parameters(), lr=dense_params['learning_rate'], weight_decay=l2_reg)

print(dense_model)

In [ ]:
dense_params

In [ ]:
if os.path.exists('dense_model.pth'):
    dense_model.load_state_dict(torch.load('dense_model.pth'))
    dense_model.to(DEVICE)
    print("Model loaded from dense_model.pth")
else:
    dense_train_dataset = get_static_dataset(dense_normalization_window, "1998-01-01", validation_start_date)
    dense_val_dataset = get_static_dataset(dense_normalization_window, validation_start_date, validation_end_date)

    dense_train_loader = DataLoader(dense_train_dataset, batch_size=dense_batch_size, shuffle=True)
    dense_val_loader = DataLoader(dense_val_dataset, batch_size=dense_batch_size, shuffle=False)
    _, dense_model = train(
        model=dense_model,
        train_loader=dense_train_loader,
        val_loader=dense_val_loader,
        criterion=loss_fn,
        optimizer=dense_optimizer,
        num_epochs=100,
        patience=10,
        l1_reg=l1_reg,
        verbose=True
    )
    torch.save(dense_model.state_dict(), 'dense_model.pth')

In [ ]:
if not os.path.exists("quant_reg.pth"):
    dense_train_dataset = get_static_dataset(dense_normalization_window, "1998-01-01", validation_start_date)

    big_X = []
    big_y = []
    for i in range(len(dense_train_dataset)):
        x, _, z, y, _ = dense_train_dataset[i]
        xx = torch.cat((x, z))
        y = y.view(-1).cpu().numpy()
        big_X.append(xx.cpu().numpy())
        big_y.append(y)
    big_X = np.array(big_X)
    big_y = np.array(big_y).reshape(-1)

    quant_reg = LinearQuantileRegression(quantiles)
    quant_reg.train(big_X, big_y)

    with open("quant_reg.pth", "wb") as f:
        pickle.dump(quant_reg, f)
else:
    with open("quant_reg.pth", "rb") as f:
        quant_reg = pickle.load(f)
    print("Model loaded from quant_reg.pth")

In [17]:
dense_test_dataset = get_test_dataset(dense_normalization_window, test_start_date, test_end_date)
lstm_test_dataset = get_test_dataset(lstm_normalization_window, test_start_date, test_end_date)

In [18]:
assets = dense_test_dataset.assets
datas = dense_test_dataset.datas

In [ ]:
batch_size = 1024
for asset in tqdm(assets, desc='Inferencing models'):
    grouping = get_grouping(datas, asset)
    if grouping not in results:
        results[grouping] = {
            'linear': 0,
            'dense': 0,
            'lstm': 0,
        }
    dense_test_dataset.set_main_asset(asset)
    lstm_test_dataset.set_main_asset(asset)

    dense_data_loader = DataLoader(dense_test_dataset, batch_size=1024, shuffle=False)
    lstm_data_loader = DataLoader(lstm_test_dataset, batch_size=1024, shuffle=False)
    dense_losses = inference(dense_model, dense_data_loader, test_loss_fn, is_dense=True)
    lstm_losses = inference(lstm_model, lstm_data_loader, test_loss_fn, is_dense=False)
    linear_loss = inference(quant_reg, dense_data_loader, test_loss_fn, is_dense=False,
                            is_linear=True)

    results[grouping]['linear'] += linear_loss.mean().item()
    results[grouping]['dense'] += dense_losses.mean().item()
    results[grouping]['lstm'] += lstm_losses.mean().item()
results = {k: {model: value / 10 for model, value in v.items()} for k, v in results.items()}
market_results = results.copy()

In [ ]:
report_results(market_results)

In [ ]:
plot_3d_combined_pdfs('NVDA', dense_model, lstm_model, quant_reg,
                      dense_test_dataset, lstm_test_dataset, is_linear=False)
plot_pdf('NVDA', dense_model, lstm_model, quant_reg,
         dense_test_dataset, lstm_test_dataset, is_linear=False)

In [ ]:
print("Generating Normally Distributed Test Dataset")
dense_normal_test_dataset, lstm_normal_test_dataset = get_test_synthetic_dataset(
    dense_normalization_window, lstm_normalization_window, 1000, 0.7, distribution='normal')
print("Generating Log Normally Distributed Test Dataset")
dense_log_normal_test_dataset, lstm_log_normal_test_dataset = get_test_synthetic_dataset(
    dense_normalization_window, lstm_normalization_window, 1000, 0.7, distribution='lognormal')
print("Generating Gamma Distributed Test Dataset")
dense_gamma_test_dataset, lstm_gamma_test_dataset = get_test_synthetic_dataset(
    dense_normalization_window, lstm_normalization_window, 1000, 0.7, distribution='gamma')
print("Generating Uniform Distributed Test Dataset")
dense_uniform_test_dataset, lstm_uniform_test_dataset = get_test_synthetic_dataset(
    dense_normalization_window, lstm_normalization_window, 1000, 0.7, distribution='uniform')

synthetic_data = {
    'Normal': (dense_normal_test_dataset, lstm_normal_test_dataset),
    'Log Normal': (dense_log_normal_test_dataset, lstm_log_normal_test_dataset),
    'Gamma': (dense_gamma_test_dataset, lstm_gamma_test_dataset),
    'Uniform': (dense_uniform_test_dataset, lstm_uniform_test_dataset)
}

In [ ]:
synthetic_results = {}
for distribution, (dense_synthetic_test_dataset, lstm_synthetic_test_dataset) in tqdm(synthetic_data.items(), desc='Inferencing synthetic datasets'):
    dense_total_loss = 0
    lstm_total_loss = 0
    for i in range(0, 10):
        dense_synthetic_test_dataset.set_main_asset(f"synthetic_{i}")
        lstm_synthetic_test_dataset.set_main_asset(f"synthetic_{i}")
        dense_data_loader = DataLoader(dense_synthetic_test_dataset, batch_size=1024, shuffle=False)
        lstm_data_loader = DataLoader(lstm_synthetic_test_dataset, batch_size=1024, shuffle=False)
        linear_losses = inference(quant_reg, dense_data_loader,
                                  test_loss_fn, is_dense=False, is_linear=True)
        dense_losses = inference(dense_model, dense_data_loader, test_loss_fn, is_dense=True)
        lstm_losses = inference(lstm_model, lstm_data_loader, test_loss_fn, is_dense=False)
        dense_total_loss += dense_losses.mean().item()
        lstm_total_loss += lstm_losses.mean().item()

    dense_total_loss /= 10
    lstm_total_loss /= 10

    results[f"{distribution} synthetic"] = {
        'linear': linear_losses.mean().item(),
        'dense': dense_total_loss,
        'lstm': lstm_total_loss
    }
    synthetic_results[f"{distribution} synthetic"] = {
        'linear': linear_losses.mean().item(),
        'dense': dense_total_loss,
        'lstm': lstm_total_loss
    }

In [ ]:
report_results(synthetic_results)

In [ ]:
report_results(results)

In [ ]:
plot_3d_combined_pdfs('synthetic_1', dense_model, lstm_model, quant_reg, dense_normal_test_dataset, lstm_normal_test_dataset, is_linear=True)
plot_pdf('synthetic_1', dense_model, lstm_model, quant_reg, dense_normal_test_dataset, lstm_normal_test_dataset, is_linear=True)

In [17]:
import numpy as np
from scipy.stats import norm, wasserstein_distance, t, ks_2samp
from typing import Dict, List, Union
import pandas as pd


def wasserstein_metric(quantiles: np.ndarray, real_returns: np.ndarray) -> float:
    """Calculate Wasserstein distance for a single prediction window."""
    real_cdf = np.sort(real_returns)
    pred_cdf = np.sort(quantiles)
    return wasserstein_distance(real_cdf, pred_cdf)


def ks_statistic(quantiles: np.ndarray, real_returns: np.ndarray) -> float:
    """Calculate KS statistic for a single prediction window."""
    return ks_2samp(quantiles, real_returns).statistic


def quantile_coverage_error(quantiles: np.ndarray, real_returns: np.ndarray,
                            quant_probs: np.ndarray) -> float:
    """Calculate QCE for a single prediction window."""
    empirical_proportions = np.array([
        np.mean(real_returns <= q) for q in quantiles
    ])
    errors = np.abs(empirical_proportions - quant_probs)
    return np.mean(errors)


def historical_baseline(real_returns: np.ndarray, quant_probs: np.ndarray) -> np.ndarray:
    """Calculate historical baseline quantiles."""
    return np.percentile(real_returns, quant_probs * 100)


def gaussian_baseline(real_returns: np.ndarray, quant_probs: np.ndarray) -> np.ndarray:
    """Calculate Gaussian baseline quantiles."""
    mu, sigma = np.mean(real_returns), np.std(real_returns)
    return norm.ppf(quant_probs, loc=mu, scale=sigma)


def student_t_baseline(real_returns: np.ndarray, quant_probs: np.ndarray) -> np.ndarray:
    """Calculate Student's t baseline quantiles."""
    df, loc, scale = 5, np.mean(real_returns), np.std(real_returns)
    return t.ppf(quant_probs, df, loc=loc, scale=scale)


def evaluate_predictions_2d(predicted_quantiles: np.ndarray, future_returns: np.ndarray,
                            observed_returns: np.ndarray, quant_probs: np.ndarray) -> Dict:
    """
    Evaluate distributional predictions across multiple time windows.

    Args:
        predicted_quantiles: Shape (n_windows, n_quantiles)
        future_returns: Shape (n_windows, n_future_steps)
        observed_returns: Shape (n_windows, n_observed_steps)
        quant_probs: Shape (n_quantiles,)

    Returns:
        Dict containing mean and std of metrics across windows
    """
    n_windows = predicted_quantiles.shape[0]
    metrics_per_window = []

    for i in range(n_windows):
        # Generate baseline predictions for this window
        hist_baseline = historical_baseline(observed_returns[i], quant_probs)
        gauss_baseline = gaussian_baseline(observed_returns[i], quant_probs)
        t_baseline = student_t_baseline(observed_returns[i], quant_probs)

        # Calculate metrics for this window
        window_metrics = {
            'qce': {
                'historical': quantile_coverage_error(hist_baseline, future_returns[i], quant_probs),
                'gaussian': quantile_coverage_error(gauss_baseline, future_returns[i], quant_probs),
                'student_t': quantile_coverage_error(t_baseline, future_returns[i], quant_probs),
                'qlstm': quantile_coverage_error(predicted_quantiles[i], future_returns[i], quant_probs)
            },
            'wasserstein': {
                'historical': wasserstein_metric(hist_baseline, future_returns[i]),
                'gaussian': wasserstein_metric(gauss_baseline, future_returns[i]),
                'student_t': wasserstein_metric(t_baseline, future_returns[i]),
                'qlstm': wasserstein_metric(predicted_quantiles[i], future_returns[i])
            },
            'ks': {
                'historical': ks_statistic(hist_baseline, future_returns[i]),
                'gaussian': ks_statistic(gauss_baseline, future_returns[i]),
                'student_t': ks_statistic(t_baseline, future_returns[i]),
                'qlstm': ks_statistic(predicted_quantiles[i], future_returns[i])
            }
        }
        metrics_per_window.append(window_metrics)

    # Calculate aggregate statistics
    aggregate_metrics = {
        'mean': {},
        'std': {},
        'min': {},
        'max': {},
        'median': {}
    }

    # Helper function to extract specific metric across windows
    def extract_metric(metrics_list, metric_path):
        values = []
        for m in metrics_list:
            curr = m
            for key in metric_path:
                curr = curr[key]
            values.append(curr)
        return np.array(values)

    # Metrics to aggregate
    metric_paths = [
        ('qce', model) for model in ['historical', 'gaussian', 'student_t', 'qlstm']
    ] + [
        ('wasserstein', model) for model in ['historical', 'gaussian', 'student_t', 'qlstm']
    ] + [
        ('ks', model) for model in ['historical', 'gaussian', 'student_t', 'qlstm']
    ]

    # Calculate statistics for each metric
    for path in metric_paths:
        values = extract_metric(metrics_per_window, path)
        metric_name = '_'.join(path)

        aggregate_metrics['mean'][metric_name] = np.mean(values)
        aggregate_metrics['std'][metric_name] = np.std(values)
        aggregate_metrics['min'][metric_name] = np.min(values)
        aggregate_metrics['max'][metric_name] = np.max(values)
        aggregate_metrics['median'][metric_name] = np.median(values)

    return aggregate_metrics


def print_aggregate_metrics(metrics: Dict):
    """Pretty print the aggregate metrics."""
    stats = ['mean', 'std', 'median', 'min', 'max']
    metric_types = ['qce', 'wasserstein', 'ks']

    for metric_type in metric_types:
        print(f"\n{metric_type.upper()} Metrics:")
        relevant_metrics = {k: v for k, v in metrics['mean'].items() if metric_type in k}

        for metric_name, mean_value in relevant_metrics.items():
            print(f"\n{metric_name}:")
            for stat in stats:
                print(f"  {stat}: {metrics[stat][metric_name]:.4f}")

In [18]:
prediction_test_set = get_test_dataset(lstm_normalization_window, test_start_date, test_end_date, lookforward=30)

In [19]:
def evaluate_market_group(
    group_name: str,
    prediction_test_set,
    lstm_model,
    device: str,
    q: np.ndarray
) -> dict:
    """
    Evaluate predictions for a market group.
    
    Args:
        group_name: Name of the market group
        prediction_test_set: Dataset containing the test data
        lstm_model: Trained LSTM model
        device: Device to run computations on
        q: Quantile probabilities
        
    Returns:
        Dict containing evaluation metrics
    """
    group_assets = [a["asset"] for a in prediction_test_set.datas[group_name]]
    observed_returns = []
    future_returns = []
    all_pred_quantiles = []
    
    # Evaluate each asset in the group
    for asset in group_assets:
        prediction_test_set.set_main_asset(asset)
        lstm_data_loader = DataLoader(prediction_test_set, batch_size=1024, shuffle=False)
        lstm_model.eval()
        
        pred_quantiles = []
        ys = []
        
        # Get predictions
        for x, s, z, y, _ in lstm_data_loader:
            x, s, z, y = x.to(device), s.to(device), z.to(device), y.to(device)
            s = s.mean(dim=1)
            
            with torch.no_grad():
                _, lstm_quantiles = lstm_model(x, s, z)
            pred_quantiles.append(lstm_quantiles.detach().cpu().numpy())
            ys.append(y.cpu().numpy())
        
        # Process predictions
        pred_quantiles = np.concatenate(pred_quantiles, axis=0)/100
        ys = np.concatenate(ys, axis=0)/100
        ys = ys.reshape(ys.shape[0], -1)
        
        # Split data
        sub_observed_returns = ys[:-30]
        sub_future_returns = ys[30:]
        sub_predicted_quantiles = pred_quantiles[30:]
        
        observed_returns.append(sub_observed_returns)
        future_returns.append(sub_future_returns)
        all_pred_quantiles.append(sub_predicted_quantiles)
    
    # Concatenate results
    observed_returns = np.concatenate(observed_returns, axis=0)
    future_returns = np.concatenate(future_returns, axis=0)
    all_pred_quantiles = np.concatenate(all_pred_quantiles, axis=0)
    
    # Calculate metrics
    return evaluate_predictions_2d(
        all_pred_quantiles, future_returns, observed_returns, q
    )

In [20]:
def evaluate_all_markets(
    market_groups: list[str],
    prediction_test_set,
    lstm_model,
    device: str,
    q: np.ndarray
) -> pd.DataFrame:
    """
    Evaluate and format results for all market groups.
    
    Returns:
        Formatted DataFrame with comparisons across markets
    """
    results_by_market = {}
    
    for group in market_groups:
        # Get metrics for this market
        metrics = evaluate_market_group(
            group, prediction_test_set, lstm_model, device, q
        )
        
        # Convert to DataFrame format
        market_results = []
        
        # LSTM metrics
        market_results.append({
            'metric': 'qLSTM_wasserstein',
            'mean': metrics['mean']['wasserstein_qlstm'],
            'std': metrics['std']['wasserstein_qlstm']
        })
        market_results.append({
            'metric': 'qLSTM_ks',
            'mean': metrics['mean']['ks_qlstm'],
            'std': metrics['std']['ks_qlstm']
        })
        market_results.append({
            'metric': 'qLSTM_qce',
            'mean': metrics['mean']['qce_qlstm'],
            'std': metrics['std']['qce_qlstm']
        })
        
        # Historical baseline metrics
        market_results.append({
            'metric': 'historical_wasserstein',
            'mean': metrics['mean']['wasserstein_historical'],
            'std': metrics['std']['wasserstein_historical']
        })
        market_results.append({
            'metric': 'historical_ks',
            'mean': metrics['mean']['ks_historical'],
            'std': metrics['std']['ks_historical']
        })
        market_results.append({
            'metric': 'historical_qce',
            'mean': metrics['mean']['qce_historical'],
            'std': metrics['std']['qce_historical']
        })
        
        # Gaussian baseline metrics
        market_results.append({
            'metric': 'gaussian_wasserstein',
            'mean': metrics['mean']['wasserstein_gaussian'],
            'std': metrics['std']['wasserstein_gaussian']
        })
        market_results.append({
            'metric': 'gaussian_ks',
            'mean': metrics['mean']['ks_gaussian'],
            'std': metrics['std']['ks_gaussian']
        })
        market_results.append({
            'metric': 'gaussian_qce',
            'mean': metrics['mean']['qce_gaussian'],
            'std': metrics['std']['qce_gaussian']
        })
        
        # Student-t baseline metrics
        market_results.append({
            'metric': 'student_t_wasserstein',
            'mean': metrics['mean']['wasserstein_student_t'],
            'std': metrics['std']['wasserstein_student_t']
        })
        market_results.append({
            'metric': 'student_t_ks',
            'mean': metrics['mean']['ks_student_t'],
            'std': metrics['std']['ks_student_t']
        })
        market_results.append({
            'metric': 'student_t_qce',
            'mean': metrics['mean']['qce_student_t'],
            'std': metrics['std']['qce_student_t']
        })
        
        results_by_market[group] = pd.DataFrame(market_results)
    
    # Format results into pretty table
    return results_by_market

In [21]:
market_groups = [
    "nikkei 225", 
    "s&p 500", 
    "euro stoxx 50", 
    "cryptocurrencies",
    "commodities",
    "currency pairs"
]

results_df = evaluate_all_markets(
    market_groups,
    prediction_test_set,
    lstm_model,
    DEVICE,
    np.array(quantiles)
)


In [22]:
def generate_metric_tables(data):
    """
    Generate comparison tables for wasserstein, ks, and qce metrics across different methods and markets,
    with an added 'total' row for each metric.

    Args:
    - data (dict): Dictionary where keys are market names and values are DataFrames with columns 'metric', 'mean', and 'std'.

    Returns:
    - dict: Dictionary of DataFrames for each metric (wasserstein, ks, qce).
    """
    metrics = ['wasserstein', 'ks', 'qce']
    tables = {}

    for metric in metrics:
        # Initialize the structure of the metric table
        metric_data = {
            'market': [],
            'qLSTM': [],
            'historical': [],
            'gaussian': [],
            'student_t': []
        }

        # Populate the table for the current metric
        for market, df in data.items():
            metric_data['market'].append(market)
            for method in ['qLSTM', 'historical', 'gaussian', 'student_t']:
                value = df.loc[df['metric'] == f'{method}_{metric}', 'mean'].values[0]
                metric_data[method].append(value)

        # Create a DataFrame for the current metric
        metric_df = pd.DataFrame(metric_data)

        # Calculate totals (mean of columns) and append as the last row
        total_row = {'market': 'total'}
        for method in ['qLSTM', 'historical', 'gaussian', 'student_t']:
            total_row[method] = metric_df[method].mean()
        metric_df = pd.concat([metric_df, pd.DataFrame([total_row])], ignore_index=True)

        # Store the table
        tables[metric] = metric_df

    return tables

In [23]:
metrics_tables = generate_metric_tables(results_df)

In [ ]:
for i, (metric, table) in enumerate(metrics_tables.items()):
    print(f"\n{metric.upper()} Metrics:")
    display(table)